In [1]:
!pip install -U sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 4.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - \ done
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125938 sha256=47b0d9ae990f239b41e5a09306048d4bf5f1152fc96e4d9410cba91b3da117b7
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence-transformers


In [2]:
import pandas as pd

data = pd.read_csv('/kaggle/input/sample-data/llm_aggregated_data_v3_sample.csv')
data.head(3)

,Unnamed: 0,COUNTRY,INITIAL_TYPIFICATION_OG,INITIAL_TYPIFICATION,FINAL_TYPIFICATION_AGENT,INITIAL_LEVEL_1,INITIAL_LEVEL_2,INITIAL_LEVEL_3,FINAL_LEVEL_1,FINAL_LEVEL_2,...,prompt_embedding,chat_embedding_v1,prompt_embedding_v2,chat_embedding_v2,grouped_label,xgb_probs,mlp_probs,rfs_probs,probs_class_order,grouped_label_lvl2
0,67681,CL,sn_org_mc.sn_org_ddbmc,sn_org_au_3.sn_org_dma,sn_org_au_3.sn_org_dma,Account User,Delete my account,Unidentified,Account User,Delete my account,...,A continuación se presenta un chat entre un cl...,[-0.01584649 -0.00231411 0.00122872 ... -0.01...,A continuación se presenta un chat entre un cl...,[-0.02121758 -0.00295413 0.00293869 ... -0.01...,Account User,[[0.9622785 0.00843728 0.00289495 0.00682525 ...,[[9.96172710e-01 1.16238664e-03 3.19813768e-05...,[[0.66666667 0.13111111 0.01333333 0.06 ...,"['Account User', 'Disagree with charge', 'Prob...",Delete my account
1,29379,AR,sn_org_tupcmr.sn_org_tupcmr1.sn_org_fgi,sn_org_pwrt_1.sn_org_fgi,sn_org_pwrt_1.sn_org_fgi,Problems with the RT,Was rude and/or disrespectful,Unidentified,Problems with the RT,Was rude and/or disrespectful,...,A continuación se presenta un chat entre un cl...,[-0.01998903 -0.01046161 -0.00613752 ... -0.01...,A continuación se presenta un chat entre un cl...,[-0.01836446 -0.0132046 -0.00464593 ... -0.00...,Problems with the RT,[[0.02960206 0.03007483 0.00333534 0.8225218 ...,[[0.06824591 0.00690485 0.00100044 0.71471884 ...,[[0.08 0.07333333 0.02 0.38666667 ...,"['Account User', 'Disagree with charge', 'Prob...",Was rude and/or disrespectful
2,110004,CO,sn_org_mp.sn_org_pc.sn_org_pq_cmp,sn_org_can_3.sn_org_can_31,sn_org_can_3.sn_org_iacan,Cancellations,Information about cancellation,Unidentified,Cancellations,Information about cancellation,...,A continuación se presenta un chat entre un cl...,[-0.01284217 0.00295121 0.00336269 ... -0.00...,A continuación se presenta un chat entre un cl...,[-0.01476855 0.00780584 0.00091963 ... -0.00...,other,[[0.06077987 0.27275527 0.01139545 0.13525787 ...,[[0.10457538 0.19842347 0.02421544 0.02247231 ...,[[0.24744186 0.20139535 0.04031008 0.14682171 ...,"['Account User', 'Disagree with charge', 'Prob...",Information about cancellation


## Generate the Triplets Dataset

In [3]:
import logging
logging.basicConfig(level=logging.INFO)
log = logging.getLogger(__name__)
labels = data.grouped_label.value_counts().keys().tolist()
log.info(labels)

In [4]:
from tqdm import tqdm
from tqdm.contrib.logging import logging_redirect_tqdm

triplets = []
with logging_redirect_tqdm():
    for i, v in tqdm(data.iterrows(), total = len(data)):
        anchor = v.prompt_embedding_v2
        label = v.grouped_label
        n_samples = len(labels) - 1
        pos_samples = data[data.grouped_label == label].sample(n_samples).prompt_embedding_v2.tolist()

        for l in labels:
            if l != label:
                pos = pos_samples.pop()
                neg = data[data.grouped_label == l].sample().iloc[0].prompt_embedding_v2
                triplets.append({'anchor': anchor, 'pos': pos, 'neg': neg})

100%|██████████| 4000/4000 [00:42<00:00, 93.32it/s]


In [5]:
triplets_df = pd.DataFrame(triplets)

In [6]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(triplets_df, test_size=0.2, random_state=0)

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [7]:
from sentence_transformers import InputExample
from tqdm import tqdm

train_examples = []
test_examples = []

with logging_redirect_tqdm():
    for i, v in tqdm(train.iterrows(), total=len(train)):
      train_examples.append(InputExample(texts=[v.anchor, v.pos, v.neg]))

    for i, v in tqdm(test.iterrows(), total=len(test)):
      test_examples.append(InputExample(texts=[v.anchor, v.pos, v.neg]))

100%|██████████| 4800/4800 [00:00<00:00, 12802.61it/s]


In [8]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(train_examples, shuffle=True, batch_size=3)
test_dataloader = DataLoader(test_examples, shuffle=True, batch_size=3)

## Load and Train the Sentence Transformer model

In [9]:
from sentence_transformers import SentenceTransformer, models

with logging_redirect_tqdm():
    model_id = "distilbert-base-uncased"
    bert = models.Transformer(model_id)
    pooler = models.Pooling(
            bert.get_word_embedding_dimension(),
            pooling_mode_mean_tokens=True)
    model = SentenceTransformer(modules=[bert, pooler])
    log.info(model)

/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so: undefined symbol: _ZTVN10tenso

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_transform.weight', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_projector.bias', 'vocab_transform.bias']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Use pytorch device: cuda
SentenceTransformer(
  (0): Transformer({'max_seq_length': 512, 'do_lower_case': False}) with Transformer model: DistilBertModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
)


In [10]:
from sentence_transformers import losses

train_loss = losses.TripletLoss(model=model)

In [11]:
num_epochs = 10

warmup_steps = int(len(train_dataloader) * num_epochs * 0.1) #10% of train data

In [12]:
from sentence_transformers import evaluation

evaluator = evaluation.TripletEvaluator(anchors=test[:500]['anchor'].tolist(), 
                            positives=test[:500]['pos'].tolist(), 
                            negatives=test[:500]['neg'].tolist(),
                            name='evaluation_metrics',
                            write_csv=True)

In [13]:
with logging_redirect_tqdm():
    model.fit(train_objectives=[(train_dataloader, train_loss)],
              epochs=num_epochs,
              warmup_steps=warmup_steps,
              show_progress_bar=True,
              evaluator=evaluator,
              evaluation_steps=500,
              output_path='/kaggle/working/models_distilled',
              save_best_model=True,
              checkpoint_path='/kaggle/working/runs',
              checkpoint_save_steps=2000,
              checkpoint_save_total_limit=10) 

Epoch:   0%|          | 0/10 [00:00<?, ?it/s]

Iteration:   0%|          | 0/6400 [00:00<?, ?it/s]

TripletEvaluator: Evaluating the model on evaluation_metrics dataset in epoch 0 after 500 steps:
Accuracy Cosine Distance:   	53.80
Accuracy Manhattan Distance:	53.40
Accuracy Euclidean Distance:	53.80

Save model to /kaggle/working/models_distilled
TripletEvaluator: Evaluating the model on evaluation_metrics dataset in epoch 0 after 1000 steps:
Accuracy Cosine Distance:   	51.40
Accuracy Manhattan Distance:	51.40
Accuracy Euclidean Distance:	51.60

TripletEvaluator: Evaluating the model on evaluation_metrics dataset in epoch 0 after 1500 steps:
Accuracy Cosine Distance:   	55.60
Accuracy Manhattan Distance:	56.00
Accuracy Euclidean Distance:	55.60

Save model to /kaggle/working/models_distilled
TripletEvaluator: Evaluating the model on evaluation_metrics dataset in epoch 0 after 2000 steps:
Accuracy Cosine Distance:   	60.60
Accuracy Manhattan Distance:	60.60
Accuracy Euclidean Distance:	60.00

Save model to /kaggle/working/models_distilled
Save model to /kaggle/working/runs/2000
Trip

Iteration:   0%|          | 0/6400 [00:00<?, ?it/s]

TripletEvaluator: Evaluating the model on evaluation_metrics dataset in epoch 1 after 500 steps:
Accuracy Cosine Distance:   	75.40
Accuracy Manhattan Distance:	75.80
Accuracy Euclidean Distance:	76.20

TripletEvaluator: Evaluating the model on evaluation_metrics dataset in epoch 1 after 1000 steps:
Accuracy Cosine Distance:   	76.00
Accuracy Manhattan Distance:	76.40
Accuracy Euclidean Distance:	76.40

TripletEvaluator: Evaluating the model on evaluation_metrics dataset in epoch 1 after 1500 steps:
Accuracy Cosine Distance:   	75.80
Accuracy Manhattan Distance:	77.00
Accuracy Euclidean Distance:	76.60

Save model to /kaggle/working/runs/8000
TripletEvaluator: Evaluating the model on evaluation_metrics dataset in epoch 1 after 2000 steps:
Accuracy Cosine Distance:   	80.40
Accuracy Manhattan Distance:	79.60
Accuracy Euclidean Distance:	79.80

Save model to /kaggle/working/models_distilled
TripletEvaluator: Evaluating the model on evaluation_metrics dataset in epoch 1 after 2500 steps:


...

TripletEvaluator: Evaluating the model on evaluation_metrics dataset in epoch 7 after 500 steps:
Accuracy Cosine Distance:   	99.20
Accuracy Manhattan Distance:	99.00
Accuracy Euclidean Distance:	99.00

TripletEvaluator: Evaluating the model on evaluation_metrics dataset in epoch 7 after 1000 steps:
Accuracy Cosine Distance:   	99.40
Accuracy Manhattan Distance:	99.40
Accuracy Euclidean Distance:	99.40

Save model to /kaggle/working/runs/46000
TripletEvaluator: Evaluating the model on evaluation_metrics dataset in epoch 7 after 1500 steps:
Accuracy Cosine Distance:   	99.40
Accuracy Manhattan Distance:	99.40
Accuracy Euclidean Distance:	99.40

TripletEvaluator: Evaluating the model on evaluation_metrics dataset in epoch 7 after 2000 steps:
Accuracy Cosine Distance:   	99.20
Accuracy Manhattan Distance:	99.40
Accuracy Euclidean Distance:	99.40

TripletEvaluator: Evaluating the model on evaluation_metrics dataset in epoch 7 after 2500 steps:
Accuracy Cosine Distance:   	99.40
Accuracy Ma

Iteration:   0%|          | 0/6400 [00:00<?, ?it/s]

TripletEvaluator: Evaluating the model on evaluation_metrics dataset in epoch 8 after 500 steps:
Accuracy Cosine Distance:   	99.00
Accuracy Manhattan Distance:	99.00
Accuracy Euclidean Distance:	99.00

Save model to /kaggle/working/runs/52000
TripletEvaluator: Evaluating the model on evaluation_metrics dataset in epoch 8 after 1000 steps:
Accuracy Cosine Distance:   	99.20
Accuracy Manhattan Distance:	99.20
Accuracy Euclidean Distance:	99.20

TripletEvaluator: Evaluating the model on evaluation_metrics dataset in epoch 8 after 1500 steps:
Accuracy Cosine Distance:   	99.40
Accuracy Manhattan Distance:	99.40
Accuracy Euclidean Distance:	99.40

TripletEvaluator: Evaluating the model on evaluation_metrics dataset in epoch 8 after 2000 steps:
Accuracy Cosine Distance:   	99.20
Accuracy Manhattan Distance:	99.20
Accuracy Euclidean Distance:	99.20

TripletEvaluator: Evaluating the model on evaluation_metrics dataset in epoch 8 after 2500 steps:
Accuracy Cosine Distance:   	99.60
Accuracy Ma

Iteration:   0%|          | 0/6400 [00:00<?, ?it/s]

Save model to /kaggle/working/runs/58000
TripletEvaluator: Evaluating the model on evaluation_metrics dataset in epoch 9 after 500 steps:
Accuracy Cosine Distance:   	100.00
Accuracy Manhattan Distance:	100.00
Accuracy Euclidean Distance:	100.00

TripletEvaluator: Evaluating the model on evaluation_metrics dataset in epoch 9 after 1000 steps:
Accuracy Cosine Distance:   	99.80
Accuracy Manhattan Distance:	99.80
Accuracy Euclidean Distance:	99.80

TripletEvaluator: Evaluating the model on evaluation_metrics dataset in epoch 9 after 1500 steps:
Accuracy Cosine Distance:   	100.00
Accuracy Manhattan Distance:	100.00
Accuracy Euclidean Distance:	100.00

TripletEvaluator: Evaluating the model on evaluation_metrics dataset in epoch 9 after 2000 steps:
Accuracy Cosine Distance:   	100.00
Accuracy Manhattan Distance:	100.00
Accuracy Euclidean Distance:	99.80

Save model to /kaggle/working/runs/60000
TripletEvaluator: Evaluating the model on evaluation_metrics dataset in epoch 9 after 2500 step

In [14]:
!zip -f models_distilled.zip /kaggle/working/models_distilled

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
	zip warning: models_distilled.zip not found or empty
